In [4]:
import geopandas as gpd
import numpy as np

lgas = gpd.read_file("/home/gonzalo/Downloads/2022 Floods Declared LGAs Shapefile FINAL/Floods_2022_Declared_LGA_FINAL.shp").to_crs("EPSG:4326")
lgas_bounds = lgas.bounds
bounds_all = np.min(lgas_bounds.minx), np.min(lgas_bounds.miny), np.max(lgas_bounds.maxx), np.max(lgas_bounds.maxy)
bounds_all

(146.86329100021769,
 -37.50516044138397,
 153.63872785102905,
 -23.244936010554465)

## Compute existing polygons

In [18]:
import os
from ml4floods.data import utils
from tqdm import tqdm

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/ml4cc-general-access_request_pays.json"
fs = utils.get_filesystem("gs://ml4cc_data_lake")

data = []
for meta_file in tqdm(fs.glob("gs://ml4cc_data_lake/0_DEV/1_Staging/operational/EMSR570/*/flood_meta/2022-03-*")):
    meta_file = f"gs://{meta_file}"
    meta = utils.read_pickle_from_gcp(meta_file)
    data.append({"geometry":meta["area_of_interest_polygon"], "ems_code":meta["ems_code"], "aoi_code":meta["aoi_code"], "date":meta["satellite date"]})
data = gpd.GeoDataFrame(data,crs="EPSG:4326")
data

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.94it/s]


,geometry,ems_code,aoi_code,date
0,"POLYGON ((153.20101 -28.91599, 153.44092 -28.9...",EMSR570,AOI01,2022-03-29 00:00:00+00:00
1,"POLYGON ((153.15602 -28.69964, 153.44520 -28.7...",EMSR570,AOI02,2022-03-29 00:00:00+00:00
2,"POLYGON ((152.87678 -28.57926, 153.13404 -28.5...",EMSR570,AOI03,2022-03-29 00:00:00+00:00
3,"POLYGON ((152.98821 -28.83102, 153.21246 -28.8...",EMSR570,AOI04,2022-03-29 00:00:00+00:00
4,"POLYGON ((153.41606 -28.74847, 153.60454 -28.7...",EMSR570,AOI05,2022-03-29 00:00:00+00:00
...,...,...,...,...
95,"POLYGON ((152.09898 -25.95961, 152.38684 -25.9...",EMSR570,QLAOI51,2022-03-29 00:00:00
96,"POLYGON ((151.69106 -26.19427, 152.04751 -26.1...",EMSR570,QLAOI52,2022-03-29 00:00:00
97,"POLYGON ((151.73009 -26.41593, 152.06565 -26.4...",EMSR570,QLAOI53,2022-03-29 00:00:00
98,"POLYGON ((151.47685 -26.09721, 151.73010 -26.1...",EMSR570,QLAOI54,2022-03-29 00:00:00


In [27]:

data[["geometry","aoi_code"]].to_file("/home/gonzalo/Downloads/australia_aois.geojson",driver="GeoJSON")

In [34]:
import itertools
from ml4floods.data.copernicusEMS.activations import generate_polygon
from shapely.geometry import Polygon

import warnings
warnings.filterwarnings('ignore', 'Geometry is in a ', UserWarning)

step = 0.20, 0.20
size = 0.21

pols_all = []
for idx, (x_left,y_bottom) in enumerate(itertools.product(np.arange(bounds_all[0], bounds_all[2], step[0]),
                                                          np.arange(bounds_all[1],bounds_all[3], step[1]))):
    bounds_iter = (x_left, y_bottom, x_left+size, y_bottom+size)
    pol = Polygon(generate_polygon(bounds_iter))
    
    
    # TODO do not include pol if it does not overlap the lgas
    lgas_overlap_polygons = lgas.intersection(pol)
    lgas_overlap_polygons = lgas_overlap_polygons[~lgas_overlap_polygons.is_empty].copy()
    if lgas_overlap_polygons.shape[0] == 0:
        # print(f"{idx} does not overlap the LGAS")
        continue
    area_overlap_lga = lgas_overlap_polygons.area.sum() / pol.area
    
    if area_overlap_lga<=.3:
        continue
    
    # do not include pol if it has high overlap with existing polygons
    aoi_overlap_polygons = data.intersection(pol)
    aoi_overlap_polygons = aoi_overlap_polygons[~aoi_overlap_polygons.is_empty].copy()
    
    area_overlap = aoi_overlap_polygons.area.sum() / pol.area
    if area_overlap >=.3:
        # print(f"Do not include {idx}")
        continue
    
    pols_all.append({"geometry": pol, "name":f"AUTOAOI{idx:04d}"})
    
    
pols_all = gpd.GeoDataFrame(pols_all,crs="EPSG:4326")
print(pols_all.shape)
pols_all.to_file("/home/gonzalo/Downloads/autoaois.geojson",driver="GeoJSON")

(663, 2)


In [14]:
pol.area
existing_aois = .is_empty

0     True
1     True
2     True
3     True
4     True
      ... 
95    True
96    True
97    True
98    True
99    True
Length: 100, dtype: bool

In [6]:
(153.3362,-28.1347) - (153.6401,-28.1467) - (153.3386,-28.3836)

(153.3362, -28.1347)